In [2]:
import pandas as pd
import numpy as np

One major thing that needs to be figured out for this dataset is how to read it all...It is over 10 million lines long. I want to explore ways to read in chunks analyze that data then read in another chunk and create a visualization that uses all the data (or at least most of it) 

In [3]:
# Line below isn't really feasible to run since the file is over 10 million lines long 
#books00_df = pd.read_csv('amazon_reviews_us_Books_v1_00.tsv', sep='\t', error_bad_lines=False)

# Broke the data into a smaller set to work with for now
books00_df = pd.read_csv('books_00_100k.tsv', sep='\t', error_bad_lines=False)

b'Skipping line 3524: expected 15 fields, saw 22\nSkipping line 5282: expected 15 fields, saw 22\nSkipping line 20478: expected 15 fields, saw 22\nSkipping line 25895: expected 15 fields, saw 22\nSkipping line 27016: expected 15 fields, saw 22\nSkipping line 59798: expected 15 fields, saw 22\n'
b'Skipping line 69198: expected 15 fields, saw 22\nSkipping line 71953: expected 15 fields, saw 22\nSkipping line 78720: expected 15 fields, saw 22\nSkipping line 81300: expected 15 fields, saw 22\nSkipping line 87683: expected 15 fields, saw 22\nSkipping line 90516: expected 15 fields, saw 22\nSkipping line 91147: expected 15 fields, saw 22\n'


error_bad_lines = False will remove any lines that have more entries per row than expected. This is the first step in the cleaning process 

In [5]:
books00_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,25933450,RJOVP071AVAJO,0439873800,84656342,There Was an Old Lady Who Swallowed a Shell!,Books,5,0.0,0.0,N,Y,Five Stars,I love it and so does my students!,2015-08-31
1,US,1801372,R1ORGBETCDW3AI,1623953553,729938122,I Saw a Friend,Books,5,0.0,0.0,N,Y,"Please buy ""I Saw a Friend""! Your children wil...",My wife and I ordered 2 books and gave them as...,2015-08-31
2,US,5782091,R7TNRFQAOUTX5,142151981X,678139048,"Black Lagoon, Vol. 6",Books,5,0.0,0.0,N,Y,Shipped fast.,Great book just like all the others in the ser...,2015-08-31
3,US,32715830,R2GANXKDIFZ6OI,014241543X,712432151,If I Stay,Books,5,0.0,0.0,N,N,Five Stars,So beautiful,2015-08-31
4,US,14005703,R2NYB6C3R8LVN6,1604600527,800572372,Stars 'N Strips Forever,Books,5,2.0,2.0,N,Y,Five Stars,Enjoyed the author's story and his quilts are ...,2015-08-31


Some questions to answer for this set of data
1. What is the date range?
2. What's the breakdown of star ratings? How many are 5's etc?
3. How many different books are there?
4. How often is the review body empty?
5. What to do about a 0 star rating? Is it real?

In [9]:
books00_df['string date'] = (books00_df['review_date']).apply(str)

In [14]:
# This shows that there are some dates that are missing so we need to clean that data 

print(books00_df['string date'].max())
print(books00_df['string date'].min())
print('Total number of nans in the review date:', books00_df['review_date'].isna().sum())

nan
2015-08-22
Total number of nans in the review date: 6


In [18]:
# Looks for nans in other columns 
print('Total number of nans in the product title:', books00_df['product_title'].isna().sum())
print('Total number of nans in the star rating:', books00_df['star_rating'].isna().sum())
print('Total number of nans in the review body:', books00_df['review_body'].isna().sum())
print('Total number of nans in the helpful votes:', books00_df['helpful_votes'].isna().sum())
print('Total number of nans in the total votes:', books00_df['total_votes'].isna().sum())
print('Total number of nans in the verified purchase:', books00_df['verified_purchase'].isna().sum())

Total number of nans in the product title: 0
Total number of nans in the star rating: 0
Total number of nans in the review body: 9
Total number of nans in the helpful votes: 2
Total number of nans in the total votes: 2
Total number of nans in the verified purchase: 2


In [19]:
df = books00_df[['product_title','review_date','review_body', 'star_rating', 'verified_purchase', 
                 'helpful_votes','total_votes']]

In [20]:
df = df.dropna()

In [22]:
print(books00_df.shape)
print(df.shape)

(97484, 16)
(97471, 7)


In [23]:
print('min date:', df['review_date'].min())
print('max date:', df['review_date'].max())

min date: 2015-08-22
max date: 2015-08-31


In [24]:
df['star_rating'].value_counts()

5    70883
4    13541
3     5880
1     4144
2     3023
Name: star_rating, dtype: int64

In [25]:
len(df['product_title'].unique())

71224

In [26]:
df['len_review_body'] = df['review_body'].apply(len)

In [27]:
df.head()

,product_title,review_date,review_body,star_rating,verified_purchase,helpful_votes,total_votes,len_review_body
0,There Was an Old Lady Who Swallowed a Shell!,2015-08-31,I love it and so does my students!,5,Y,0.0,0.0,34
1,I Saw a Friend,2015-08-31,My wife and I ordered 2 books and gave them as...,5,Y,0.0,0.0,364
2,"Black Lagoon, Vol. 6",2015-08-31,Great book just like all the others in the ser...,5,Y,0.0,0.0,50
3,If I Stay,2015-08-31,So beautiful,5,N,0.0,0.0,12
4,Stars 'N Strips Forever,2015-08-31,Enjoyed the author's story and his quilts are ...,5,Y,2.0,2.0,86


In [29]:
print('min review body len:', df['len_review_body'].min())

min review body len: 1


In [30]:
print('sum of min review body len:', sum(df['len_review_body'] == 1))

sum of min review body len: 32


In [31]:
print('sum of <= 5 review body len:', sum(df['len_review_body'] <= 5))

sum of <= 5 review body len: 2135


In [32]:
print('sum of <= 3 review body len:', sum(df['len_review_body'] <= 3))

sum of <= 3 review body len: 411


I'm thinking that we may want to drop the rows that have too short of a review body length because those lines will not be interesting enough or important enough to keep.